In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.rnn = nn.LSTM(input_size=28, hidden_size=64, batch_first=True)
        self.batchnorm = nn.BatchNorm1d(64)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(64, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, input):
        input = input.reshape(-1, 28, 28)
        output, hidden = self.rnn(input)
        output = output[:, -1, :]
        output = self.batchnorm(output)
        output = self.dropout1(output)
        output = self.fc1(output)
        output = F.relu(output)
        output = self.dropout2(output)
        output = self.fc2(output)
        output = F.log_softmax(output, dim=1)
        return output

In [3]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))

In [5]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:
use_cuda = torch.cuda.is_available()
torch.manual_seed(45123)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=1000, shuffle=True, **kwargs)

model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=0.1)

scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
for epoch in range(1, 10):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

torch.save(model.state_dict(), "mnist_rnn.pt")

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data\MNIST\raw\train-images-idx3-ubyte.gz to data\MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data\MNIST\raw\train-labels-idx1-ubyte.gz to data\MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

...

  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data\MNIST\raw\t10k-labels-idx1-ubyte.gz to data\MNIST\raw

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.379921


C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\functional.py:1331: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Train Epoch: 1 [640/60000 (1%)]	Loss: 2.306254
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.269907
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.258081
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.292195
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.191405
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.175229
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.084040
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.146000
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.135360
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.111367
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.002632
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.994636
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.912580
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.954830
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.906932
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.832448
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.935462
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.844297
Train Epoch: 1 [12160/60000 (20%)]	Loss: 1.873972
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.864074
Train Epoch: 1 [13440/60

KeyboardInterrupt: 

In [ ]:
# MasoudKaviani.ir